In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [ ]:
data = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
data.head(15)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.corr()

In [ ]:
data1 = data.copy()
data1['quality_map'] = data['quality'].apply(lambda x:0 if x<4 else (1 if x>3 and x<7 else 2))
data1.head(15)

In [ ]:
data1.info()

# DATA VISUALISATION

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data1['quality_map'])

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='fixed acidity',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='volatile acidity',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(y='volatile acidity',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='citric acid',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(y='citric acid',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='residual sugar',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='chlorides',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(y='chlorides',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='free sulfur dioxide',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='total sulfur dioxide',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='density',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='pH',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='sulphates',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(y='sulphates',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(y='alcohol',x='quality_map',data=data1)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(data1.corr())

# DATA PREPARATION AND MODELLING

In [ ]:
data1.info()

In [ ]:
features = ['fixed acidity','volatile acidity','citric acid','chlorides','free sulfur dioxide',
           'total sulfur dioxide','sulphates','alcohol','residual sugar','pH','density']
features1 = ['fixed acidity','citric acid','sulphates','alcohol']
X = data1[features]
y = data1['quality_map']
print(X.shape,y.shape)

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
rf = RandomForestClassifier(bootstrap=True,random_state=42,max_depth=35,
                           n_estimators=2500,n_jobs=-1)
rf.fit(X_train,y_train)
accuracy_score(y_test,rf.predict(X_test))

In [ ]:
xgb = XGBClassifier( booster='gbtree', colsample_bylevel=1,colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=4, n_estimators=4500,
             n_jobs=4, nthread=None, 
             reg_alpha=0.6, reg_lambda=0.6,
             silent=None, subsample=0.8, verbosity=1)
xgb.fit(X_train,y_train)
accuracy_score(y_test,xgb.predict(X_test))

In [ ]:
pred = xgb.predict(X_test)
pred

In [ ]:
y_test.value_counts()

In [ ]:
output = pd.DataFrame({'Actual_quality': y_test,'Predicted_quality': pred})
output.head(15)

In [ ]:
output['Actual_quality'] = output['Actual_quality'].apply(lambda x:'Bad' if x==0 else('Average' if x==1 else 'Good'))
output['Predicted_quality'] = output['Predicted_quality'].apply(lambda x:'Bad' if x==0 else('Average' if x==1 else 'Good'))

In [ ]:
output.head(15)

In [ ]:
output.to_csv('Wine_predictions.csv',index=True)